In [ ]:
import tensorflow as tf
import numpy as np

class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [ ]:
class MLP(tf.keras.Model):
  def __init__(self):
      super().__init__()
      self.flatten = tf.keras.layers.Flatten()    # Flatten層將除第一維（batch_size）以外的維度展平
      self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
      self.dense2 = tf.keras.layers.Dense(units=10)

  @tf.function
  def call(self, inputs):         # [batch_size, 28, 28, 1]
      x = self.flatten(inputs)    # [batch_size, 784]
      x = self.dense1(x)          # [batch_size, 100]
      x = self.dense2(x)          # [batch_size, 10]
      output = tf.nn.softmax(x)
      return output

**TensorBoard：訓練過程可視化**

有時，你希望查看模型訓練過程中各個參數的變化情況（例如損失函數 Loss 的值）。雖然可以通過命令行輸出來查看，但有時顯得不夠直觀。而 TensorBoard 就是一個能夠幫助我們將訓練過程可視化的工具。

**實時查看參數變化情況**

首先在程式碼目錄下建立一個文件夾（如 ./tensorboard ）存放 TensorBoard 的記錄文件，並在程式碼中實例化一個記錄器：

summary_writer = tf.summary.create_file_writer('./tensorboard')     # 參數為記錄文件所保存的目錄

In [ ]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# writer
summary_writer = tf.summary.create_file_writer('./tensorboard') 
tf.summary.trace_on(graph=True, profiler=True)  # 開啟Trace，可以記錄圖結構和profile資訊

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))

        with summary_writer.as_default():                               # 希望使用的記錄器
          tf.summary.scalar("loss", loss, step=batch_index)

    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)    # 保存Trace資訊到文件

串流輸出內容已截斷至最後 5000 行。
batch 1001: loss 0.103742
batch 1002: loss 0.087970
batch 1003: loss 0.061048
batch 1004: loss 0.072350
batch 1005: loss 0.122419
batch 1006: loss 0.172757
batch 1007: loss 0.139443
batch 1008: loss 0.233112
batch 1009: loss 0.222251
batch 1010: loss 0.201392
batch 1011: loss 0.174509
batch 1012: loss 0.251577
batch 1013: loss 0.142532
batch 1014: loss 0.104344
batch 1015: loss 0.264567
batch 1016: loss 0.161566
batch 1017: loss 0.194765
batch 1018: loss 0.282494
batch 1019: loss 0.061361
batch 1020: loss 0.083790
batch 1021: loss 0.144079
batch 1022: loss 0.091514
batch 1023: loss 0.144283
batch 1024: loss 0.323580
batch 1025: loss 0.205611
batch 1026: loss 0.156685
batch 1027: loss 0.118413
batch 1028: loss 0.191327
batch 1029: loss 0.123055
batch 1030: loss 0.169683
batch 1031: loss 0.323260
batch 1032: loss 0.235345
batch 1033: loss 0.087220
batch 1034: loss 0.097160
batch 1035: loss 0.179883
batch 1036: loss 0.137699
batch 1037: loss 0.161791
batch 1038: loss 

當我們要對訓練過程可視化時，在程式碼目錄打開終端（如需要的話進入 TensorFlow 的 conda 環境），運行:

In [ ]:
!tensorboard --logdir=./tensorboard

2020-09-29 10:41:05.357540: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.3.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


然後使用瀏覽器訪問命令行程式所輸出的網址（一般是 http://name-of-your-computer:6006），即可訪問 TensorBoard 的可視界面

In [ ]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.975300
